In [18]:
pip install --upgrade huggingface_hub

  Obtaining dependency information for huggingface_hub from https://files.pythonhosted.org/packages/0b/05/31b21998f68c31e7ffcc27ff08531fb9af5506d765ce8d661fb0036e6918/huggingface_hub-0.24.5-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.24.5-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.24.5-py3-none-any.whl (417 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.23.4
    Uninstalling huggingface-hub-0.23.4:
      Successfully uninstalled huggingface-hub-0.23.4
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
exo 0.0.1 requires huggingface-hub==0.23.4, but you have huggingface-hub 0.24.5 which is incompatible.
exo 0.0.1 requires numpy==2.0.0, but you have numpy 1.26.4 which is incompatible.
exo 0.0.1 requires protobuf==5.27.1, but you have protobuf 4.25.4 which is incompatible.


In [ ]:
pip install tf-keras

In [ ]:
pip install transformers==4.12.3

In [13]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
import lightgbm as lgb
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Данила\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
import random

def generate_synthetic_reviews(num_reviews=1000):
    positive_reviews = [
        "This movie was amazing! I loved it.",
        "A great film with stunning visuals.",
        "Excellent plot and great acting.",
        "I thoroughly enjoyed this movie.",
        "A fantastic experience from start to finish."
    ]
    
    negative_reviews = [
        "This was a terrible movie.",
        "I hated this film. It was awful.",
        "Poor plot and terrible acting.",
        "I did not enjoy this movie at all.",
        "A waste of time and money."
    ]
    
    reviews = []
    sentiments = []
    
    for _ in range(num_reviews // 2):
        reviews.append(random.choice(positive_reviews))
        sentiments.append(1)
        reviews.append(random.choice(negative_reviews))
        sentiments.append(0)
    
    return pd.DataFrame({'review': reviews, 'sentiment': sentiments})

synthetic_data = generate_synthetic_reviews()
synthetic_data.head()


,review,sentiment
0,A fantastic experience from start to finish.,1
1,Poor plot and terrible acting.,0
2,This movie was amazing! I loved it.,1
3,I hated this film. It was awful.,0
4,A fantastic experience from start to finish.,1


In [17]:
data = synthetic_data

In [19]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Преобразование текста в нижний регистр и удаление стоп-слов
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

data['review'] = data['review'].apply(preprocess_text)
data['sentiment'] = data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Данила\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
X = data['review']
y = data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from transformers import BertTokenizer, TFBertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

def bert_encode(texts, tokenizer, max_len=128):
    all_input_ids = []
    all_attention_masks = []

    for text in texts:
        inputs = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_token_type_ids=False,
            truncation=True
        )
        all_input_ids.append(inputs['input_ids'])
        all_attention_masks.append(inputs['attention_mask'])

    return np.array(all_input_ids), np.array(all_attention_masks)

X_train_ids, X_train_masks = bert_encode(X_train, tokenizer)
X_test_ids, X_test_masks = bert_encode(X_test, tokenizer)

def create_bert_embedding(ids, masks):
    bert_outputs = bert_model([ids, masks])
    return bert_outputs.last_hidden_state[:, 0, :].numpy()

X_train_bert = create_bert_embedding(X_train_ids, X_train_masks)
X_test_bert = create_bert_embedding(X_test_ids, X_test_masks)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

  Obtaining dependency information for transformers==4.12.3 from https://files.pythonhosted.org/packages/2b/db/9145468493530a60046e59fc2a72b974591e38c1f3fc1b6c75112a91e492/transformers-4.12.3-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/56.6 kB ? eta -:--:--
     ------------- ------------------------ 20.5/56.6 kB 330.3 kB/s eta 0:00:01
     -------------------- ----------------- 30.7/56.6 kB 330.3 kB/s eta 0:00:01
     ---------------------------------- --- 51.2/56.6 kB 327.7 kB/s eta 0:00:01
     -------------------------------------- 56.6/56.6 kB 269.3 kB/s eta 0:00:00
  Obtaining dependency information for sacremoses from https://files.pythonhosted.org/packages/0b/f0/89ee2bc9da434bd78464f288fdb346bc2932f2ee80a90b2a4bbbac262c74/sacremoses-0.1.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/212.7 kB ? eta -:--:--
     ------- -------------------------------- 41.0/212.7 kB ? eta -:--:--
     --------------------- -----------

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [51 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\tokenizers
  copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers
  creating build\lib.win-amd64-cpython-311\tokenizers\models
  copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\models
  creating build\lib.win-amd64-cpython-311\tokenizers\decoders
  copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\decoders
  creating build\lib.win-amd64-cpython-311\tokenizers\normalizers
  copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\normalizers
  creating build\lib.win-amd64-cpython-311\tokenizers\pre_tokenizers
  copying py_src\tokenizers\pre_tokenizers\__init__.py -> build\lib.win-amd64-cpython-31

In [ ]:
lgbm_model = lgb.LGBMClassifier()
lgbm_model.fit(X_train_bert, y_train)
y_pred_lgbm = lgbm_model.predict(X_test_bert)

print(f"LightGBM Accuracy: {accuracy_score(y_test, y_pred_lgbm)}")
print(f"LightGBM Precision: {precision_score(y_test, y_pred_lgbm)}")
print(f"LightGBM Recall: {recall_score(y_test, y_pred_lgbm)}")


In [ ]:
# Распределение отзывов
plt.figure(figsize=(10, 6))
sns.countplot(data['sentiment'])
plt.title('Sentiment Distribution')
plt.show()

# Примеры отзывов
print(data['review'].head())
print(data['sentiment'].head())
